In [26]:
# https://www.dati.lombardia.it/Mobilit-e-trasporti/Comune-Bergamo-Incidenti-stradali/hds3-pqjq

from urllib.request import urlopen
from json import loads

data = loads(urlopen('https://www.dati.lombardia.it/resource/hds3-pqjq.json?$limit=10000').read())

In [27]:
len(data), data[0]

(10000,
 {'protocollo': '2017/001775',
  'anno': '2017',
  'data': '2017-12-31T00:00:00.000',
  'ora': '23:55',
  'localita': 'BERGAMO VIALE PAPA GIOVANNI XXIII chiesa delle grazie',
  'naturaincidente': 'Tamponamento',
  'n_illesi': '3',
  'n_feriti': '0',
  'n_riservata': '0',
  'n_morti': '0',
  'pedoni': False,
  'velocipedi': False,
  'ciclomotori_motocicli': False,
  'mezzi_pesanti': False,
  'localizzazione': {'latitude': '45.6938', 'longitude': '9.6712'},
  ':@computed_region_6hky_swhk': '5',
  ':@computed_region_ttgh_9sm5': '5'})

In [28]:
import pandas as pd
from operator import itemgetter

LOC = itemgetter('localizzazione')
N_KEYS = frozenset(k for k in data[0].keys() if k.startswith('n_'))

def extract(dct):
    res = {'data': pd.to_datetime(dct['data'][:11]+dct['ora'])}
    res.update(LOC(dct))
    res.update({k: dct[k] for k in N_KEYS})
    return res

lat_lon = pd.DataFrame(map(extract, data))
for k in ['latitude', 'longitude'] + list(N_KEYS): 
    lat_lon[k] = pd.to_numeric(lat_lon[k])

lat_lon.head()

,data,latitude,longitude,n_illesi,n_feriti,n_riservata,n_morti
0,2017-12-31 23:55:00,45.6938,9.6712,3,0,0,0
1,2017-12-31 20:20:00,45.7048,9.6929,5,0,0,0
2,2017-12-30 17:40:00,45.6889,9.6704,4,0,0,0
3,2017-12-30 13:20:00,45.6896,9.6692,4,1,0,0
4,2017-12-29 14:15:00,45.6755,9.6747,2,5,0,0


In [29]:
lat_lon['data'].describe()

count                   10000
unique                   9854
top       2016-10-14 08:50:00
freq                        3
first     2012-04-25 08:50:00
last      2017-12-31 23:55:00
Name: data, dtype: object

In [30]:
CENTER = lat_lon[['latitude', 'longitude']].mean().values.tolist()
CENTER

[45.691430510000096, 9.671049580000043]

In [44]:
import folium
from folium.plugins import FastMarkerCluster

base_map = folium.Map(location = CENTER, zoom_start = 12)
FastMarkerCluster(
    data = lat_lon[['latitude', 'longitude']].values.tolist(),
).add_to(base_map)
base_map

In [49]:
from folium.plugins import HeatMap

base_map = folium.Map(location = CENTER, zoom_start = 12)
HeatMap(
    data = lat_lon[['latitude', 'longitude', 'n_morti']].values.tolist(), 
    radius = 12, 
    max_zoom = 16
).add_to(base_map)
base_map

In [50]:
from folium.plugins import HeatMapWithTime

base_map = folium.Map(location = CENTER, zoom_start = 12)

lat_lon_months = []
for month in lat_lon['data'].dt.month.sort_values().unique():
    lat_lon_months.append(lat_lon[lat_lon['data'].dt.month == month][['latitude', 'longitude']].values.tolist())

In [55]:
base_map = folium.Map(location = CENTER, zoom_start = 14)

HeatMapWithTime(
    data = lat_lon_months, 
    radius = 16, 
).add_to(base_map)
base_map